#### Dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import nfl_data_py as nfl

The goal of this iteration is to train on a model on the optimal path a defender could take to succesfully tackle a ball carrier.

Features:

- Ball carrier metrics
- Tackler (or group of tackler) metrics
- Non-ball carrier metrics
- Distance to sideline

### Loading the Data

In [2]:
# Turn off pandas column limit
pd.set_option('display.max_columns', None)

In [3]:
# Read in the data
df = pd.read_csv("data/tracking_week_1.csv")

In [5]:
df.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN


#### Cleaning and Preparing the Data

Remove any rows prior to the ball arriving with the ball carrier

In [6]:
# Value count of event
df['event'].value_counts()

event
first_contact                28773
tackle                       26928
ball_snap                    16415
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: count, dtype: int64